# T1543.001 - Create or Modify System Process: Launch Agent
Adversaries may create or modify launch agents to repeatedly execute malicious payloads as part of persistence. When a user logs in, a per-user launchd process is started which loads the parameters for each launch-on-demand user agent from the property list (.plist) file found in <code>/System/Library/LaunchAgents</code>, <code>/Library/LaunchAgents</code>, and <code>~/Library/LaunchAgents</code>.(Citation: AppleDocs Launch Agent Daemons)(Citation: OSX Keydnap malware) (Citation: Antiquated Mac Malware) Property list files use the <code>Label</code>, <code>ProgramArguments </code>, and <code>RunAtLoad</code> keys to identify the Launch Agent's name, executable location, and execution time.(Citation: OSX.Dok Malware) Launch Agents are often installed to perform updates to programs, launch user specified programs at login, or to conduct other developer tasks.

 Launch Agents can also be executed using the [Launchctl](https://attack.mitre.org/techniques/T1569/001) command.
 
Adversaries may install a new Launch Agent that executes at login by placing a .plist file into the appropriate folders with the <code>RunAtLoad</code> or <code>KeepAlive</code> keys set to <code>true</code>.(Citation: Sofacy Komplex Trojan)(Citation: Methods of Mac Malware Persistence) The Launch Agent name may be disguised by using a name from the related operating system or benign software. Launch Agents are created with user level privileges and execute with user level permissions.(Citation: OSX Malware Detection)(Citation: OceanLotus for OS X) 

## Atomic Tests

### Atomic Test #1 - Launch Agent
Create a plist and execute it

**Supported Platforms:** macos
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: The shared library must exist on disk at specified location (#{path_malicious_plist})

##### Check Prereq Commands:
```bash
if [ -f $PathToAtomicsFolder/T1543.001/src/atomicredteam_T1543_001.plist ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```bash
echo "The shared library doesn't exist. Check the path"; exit 1;

```

In [ ]:
Invoke-AtomicTest T1543.001 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
if [ ! -d ~/Library/LaunchAgents ]; then mkdir ~/Library/LaunchAgents; fi;
sudo cp $PathToAtomicsFolder/T1543.001/src/atomicredteam_T1543_001.plist ~/Library/LaunchAgents/com.atomicredteam.plist
sudo launchctl load -w ~/Library/LaunchAgents/com.atomicredteam.plist
```

In [ ]:
Invoke-AtomicTest T1543.001 -TestNumbers 1

### Atomic Test #2 - Event Monitor Daemon Persistence
This test adds persistence via a plist to execute via the macOS Event Monitor Daemon. 

**Supported Platforms:** macos
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `bash`
```bash
sudo cp $PathToAtomicsFolder/T1543.001/src/atomicredteam_T1543_001.plist /etc/emond.d/rules/atomicredteam_T1543_001.plist
sudo touch /private/var/db/emondClients/randomflag
```

In [ ]:
Invoke-AtomicTest T1543.001 -TestNumbers 2

## Detection
Monitor Launch Agent creation through additional plist files and utilities such as Objective-See’s  KnockKnock application. Launch Agents also require files on disk for persistence which can also be monitored via other file monitoring applications.

Ensure Launch Agent's <code> ProgramArguments </code> key pointing to executables located in the <code>/tmp</code> or <code>/shared</code> folders are in alignment with enterprise policy. Ensure all Launch Agents with the <code>RunAtLoad</code> key set to <code>true</code> are in alignment with policy. 